In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time


In [2]:
raw_df=pd.read_csv(r"C:\Users\amith\Downloads\ofqual_details_20250127.csv\ofqual_details_20250127.csv")
# Filter rows where status is "Available to learners"
new_df = raw_df[raw_df['status'] == "Available to learners"]


In [9]:
from urllib.parse import urlparse
import pandas as pd


# Create a new column for the domain using netloc
new_df['domain'] = new_df['specification'].apply(lambda x: urlparse(x).netloc if pd.notnull(x) else '')

# Filter rows where domain matches
cng_df = new_df[new_df['domain'] == "www.cityandguilds.com"]

cng_df


C:\Users\amith\AppData\Local\Temp\ipykernel_19332\1708387515.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['domain'] = new_df['specification'].apply(lambda x: urlparse(x).netloc if pd.notnull(x) else '')


,qualification_type,qualification_level,qualification_number,assessment_methods,sector_subject_area,awarding_organisation,grading_type,total_credits,funding_in_england,status,...,european_qualification_level,ofqual_id,guided_learning_hours,total_qualification_time,specification,specialisms,optional_routes,grading_scale,end-point_assessment_standard,domain
12582,Occupational Qualification,Level 2,500/8317/X,Portfolio of Evidence,Horticulture and forestry,/organisations/RN5217,Pass/Fail,47.0,https://www.qualifications.education.gov.uk/Qu...,Available to learners,...,Level 3,500/8317/X,340.0,470.0,http://www.cityandguilds.com/qualifications-an...,NaN,NaN,NaN,NaN,www.cityandguilds.com
12586,Vocationally-Related Qualification,Level 2,501/1789/0,"E-assessment, Multiple Choice Examination, Por...",Digital technology (practitioners),/organisations/RN5217,Pass/Fail,48.0,https://www.qualifications.education.gov.uk/Qu...,Available to learners,...,Level 3,501/1789/0,270.0,480.0,http://www.cityandguilds.com/qualifications-an...,NaN,NaN,NaN,NaN,www.cityandguilds.com
12600,Vocationally-Related Qualification,Level 3,501/1788/9,"Coursework, E-assessment, Multiple Choice Exam...",Digital technology (practitioners),/organisations/RN5217,Pass/Fail,72.0,https://www.qualifications.education.gov.uk/Qu...,Available to learners,...,Level 4,501/1788/9,525.0,720.0,http://www.cityandguilds.com/qualifications-an...,NaN,NaN,NaN,NaN,www.cityandguilds.com
12615,Vocationally-Related Qualification,Level 3,500/3476/5,"Coursework, E-assessment, Multiple Choice Exam...",Digital technology (practitioners),/organisations/RN5217,Pass/Fail,24.0,https://www.qualifications.education.gov.uk/Qu...,Available to learners,...,Level 4,500/3476/5,95.0,240.0,https://www.cityandguilds.com/qualifications-a...,NaN,NaN,NaN,NaN,www.cityandguilds.com
12617,Vocationally-Related Qualification,Level 2,501/1430/X,"Coursework, E-assessment, Multiple Choice Exam...",Digital technology (practitioners),/organisations/RN5217,Pass/Fail,37.0,https://www.qualifications.education.gov.uk/Qu...,Available to learners,...,Level 3,501/1430/X,250.0,370.0,http://www.cityandguilds.com/qualifications-an...,NaN,NaN,NaN,NaN,www.cityandguilds.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14986,English For Speakers of Other Languages,Entry Level,601/4187/6,"Multiple Choice Examination, Practical Demonst...",Foundations for learning and life,/organisations/RN5217,Pass/Fail,10.0,https://www.qualifications.education.gov.uk/Qu...,Available to learners,...,NaN,601/4187/6,90.0,100.0,https://www.cityandguilds.com/qualifications-a...,NaN,NaN,NaN,NaN,www.cityandguilds.com
14987,Vocationally-Related Qualification,Level 1,601/5120/1,Practical Demonstration/Assignment,Engineering,/organisations/RN5217,Pass/Fail,7.0,https://www.qualifications.education.gov.uk/Qu...,Available to learners,...,Level 2,601/5120/1,60.0,70.0,http://www.cityandguilds.com/qualifications-an...,NaN,NaN,NaN,NaN,www.cityandguilds.com
14996,Vocationally-Related Qualification,Level 1,500/8862/2,"Oral Examination, Practical Demonstration/Assi...",Transportation operations and maintenance,/organisations/RN5217,Pass/Fail,45.0,https://www.qualifications.education.gov.uk/Qu...,Available to learners,...,Level 2,500/8862/2,352.0,450.0,http://www.cityandguilds.com/qualifications-an...,NaN,NaN,NaN,NaN,www.cityandguilds.com
15001,Vocationally-Related Qualification,Level 3,500/8658/3,Practical Demonstration/Assignment,Service enterprises,/organisations/RN5217,Pass/Fail,4.0,https://www.qualifications.education.gov.uk/Qu...,Available to learners,...,Level 4,500/8658/3,37.0,40.0,http://www.cityandguilds.com/qualifications-an...,NaN,NaN,NaN,NaN,www.cityandguilds.com


Extract URLS from searching ofqual ids

In [15]:

def scrape_cityandguilds_links(ids):

    base_url = "https://www.cityandguilds.com/search?n=0&q={}&t=courses+and+apprenticeships&s=relevance"
    all_links = []
    
    # User agent to mimic a browser
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    for id in ids:
        search_id = id[4:8]
        print(f"Processing ID: {id}")
        url = base_url.format(search_id)
        
        try:
            # Send request with a timeout
            response = requests.get(url, headers=headers, timeout=10)
            
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                
                # Find all elements with class "results-list"
                results_lists = soup.find_all(class_="results-list")
                
                if results_lists:
                    for results_list in results_lists:
                        # Find all <a> tags that are direct children of <li> elements
                        # This avoids links inside <p> tags
                        for li in results_list.find_all('li', recursive=False):
                            # Get direct <a> tags (not inside <p>)
                            for a in li.find_all('a', recursive=False):
                                link = a.get('href')
                                if link:
                                    # Convert relative URLs to absolute
                                    if link.startswith('/'):
                                        link = f"https://www.cityandguilds.com{link}"
                                    all_links.append(link)
                            
                            # Also look for <a> tags that are not inside <p>
                            for div in li.find_all('div', recursive=False):
                                for a in div.find_all('a', recursive=True):
                                    # Check if the <a> tag is not inside a <p> tag
                                    if not a.find_parent('p'):
                                        link = a.get('href')
                                        if link:
                                            # Convert relative URLs to absolute
                                            if link.startswith('/'):
                                                link = f"https://www.cityandguilds.com{link}"
                                            all_links.append(link)
                    
                else:
                    print(f"No results list found for ID {id}")
            else:
                print(f"Failed to retrieve page for ID {id}. Status code: {response.status_code}")
        
        except Exception as e:
            print(f"Error processing ID {id}: {str(e)}")
        
        # Add a short delay to avoid overloading the server
        time.sleep(1)
    
    return all_links

ids = cng_df['ofqual_id'].tolist()
# Scrape links for each ID
all_links = scrape_cityandguilds_links(ids)

# Print all links
print("\nAll links:")
for link in all_links:
    print(link)

# Optionally, create a simple DataFrame with just the links
links_df = pd.DataFrame(all_links, columns=['url'])
links_df.to_csv("cng_links.csv", index=False)
print(f"\nTotal links found: {len(all_links)}")
    

Processing ID: 500/8317/X
Processing ID: 501/1789/0
Processing ID: 501/1788/9
Processing ID: 500/3476/5
Processing ID: 501/1430/X
Processing ID: 600/2839/7
Processing ID: 500/4767/X
Processing ID: 500/4739/5
Processing ID: 500/6805/2
Processing ID: 601/3714/9
Processing ID: 601/3709/5
Processing ID: 601/3708/3
Processing ID: 500/6900/7
Processing ID: 601/2631/0
Processing ID: 500/6255/4
Processing ID: 600/7616/1
Processing ID: 500/7470/2
Processing ID: 500/7469/6
Processing ID: 500/6815/5
Processing ID: 500/5729/7
Processing ID: 500/9062/8
Processing ID: 500/9054/9
Processing ID: 600/6970/3
Processing ID: 500/6346/7
Processing ID: 500/6433/2
Processing ID: 501/0400/7
Processing ID: 500/7738/7
Processing ID: 500/6660/2
Processing ID: 500/6792/8
Processing ID: 500/6769/2
Processing ID: 500/6724/2
Processing ID: 500/6688/2
Processing ID: 500/6712/6
Processing ID: 501/1623/X
Processing ID: 500/6513/0
Processing ID: 600/3755/6
Processing ID: 600/1197/X
Processing ID: 600/1199/3
Processing I

In [20]:
links_df

,url
0,https://www.cityandguilds.com/qualifications-a...
1,https://www.cityandguilds.com/qualifications-a...
2,https://www.cityandguilds.com/qualifications-a...
3,https://www.cityandguilds.com/qualifications-a...
4,https://www.cityandguilds.com/qualifications-a...
...,...
575,https://www.cityandguilds.com/qualifications-a...
576,https://www.cityandguilds.com/qualifications-a...
577,https://www.cityandguilds.com/qualifications-a...
578,https://www.cityandguilds.com/qualifications-a...


In [21]:
links_df.drop_duplicates(inplace=True)


In [22]:
links_df

,url
0,https://www.cityandguilds.com/qualifications-a...
1,https://www.cityandguilds.com/qualifications-a...
2,https://www.cityandguilds.com/qualifications-a...
4,https://www.cityandguilds.com/qualifications-a...
6,https://www.cityandguilds.com/qualifications-a...
...,...
562,https://www.cityandguilds.com/qualifications-a...
564,https://www.cityandguilds.com/qualifications-a...
567,https://www.cityandguilds.com/qualifications-a...
569,https://www.cityandguilds.com/qualifications-a...


Extract PDF links from the extracted urls


In [27]:
def extract_pdf_links(url):
    """Append '#tab=documents' to the URL, visit the page, and return a list of PDF links."""
    pdf_links = []
    document_url = f"{url}#tab=documents"
    try:
        response = requests.get(document_url, timeout=15)
        if response.status_code != 200:
            return []  # Skip if not a successful response

        soup = BeautifulSoup(response.text, 'html.parser')
        for a_tag in soup.find_all('a'):
            href = a_tag.get('href')
            if href and (href.lower().endswith('.pdf') or '.pdf?' in href.lower()):
                # Convert relative URLs to absolute if needed
                if href.startswith('/'):
                    href = f"https://www.cityandguilds.com{href}"
                pdf_links.append(href)
    except Exception:
        # If any error occurs, simply skip this URL
        return []

    return pdf_links


# List to collect all PDF links
all_pdf_links = []

# Process each URL
for url in links_df['url']:
    pdfs = extract_pdf_links(url)
    if pdfs:  # If any PDFs were found, add them to our list
        all_pdf_links.extend(pdfs)
    time.sleep(1)  # Small delay to avoid overloading the server

# Dump the PDF links into a CSV if any are found
if all_pdf_links:
    pdf_df = pd.DataFrame({'pdf_url': all_pdf_links})
    pdf_df.to_csv('cityandguilds_pdfs.csv', index=False)
    pdf_df.to_excel('cityandguilds_pdfs.xlsx', index=False)
    print(f"Extracted {len(all_pdf_links)} PDF links and saved to cityandguilds_pdfs.csv")
else:
    print("No PDF links found.")




Extracted 7002 PDF links and saved to cityandguilds_pdfs.csv


In [28]:
len(set(all_pdf_links))


6992

In [2]:
import pandas as pd
pdf_df=pd.read_excel(r"C:\Users\amith\Kenpath\OFQUAL\cityandguilds_pdfs.xlsx")
pdf_df

,pdf_url
0,https://www.cityandguilds.com/-/media/productd...
1,https://www.cityandguilds.com/-/media/productd...
2,https://www.cityandguilds.com/-/media/productd...
3,https://www.cityandguilds.com/-/media/productd...
4,https://www.cityandguilds.com/-/media/productd...
...,...
6997,https://www.cityandguilds.com/-/media/productd...
6998,https://www.cityandguilds.com/-/media/productd...
6999,https://www.cityandguilds.com/-/media/productd...
7000,https://www.cityandguilds.com/-/media/productd...


In [5]:
import os
import requests
import hashlib
import urllib.parse
from concurrent.futures import ThreadPoolExecutor, as_completed

def download_pdf(pdf_url, save_dir, seen_urls, seen_files):
    """Download a single PDF and save it if it's unique."""
    if pdf_url in seen_urls:
        return f"Skipped (duplicate URL): {pdf_url}"
    
    seen_urls.add(pdf_url)
    
    try:
        response = requests.get(pdf_url, stream=True)
        response.raise_for_status()
        
        # Read content in chunks to avoid memory overload
        pdf_content = response.content
        
        # Generate hash for duplicate checking
        pdf_hash = hashlib.md5(pdf_content).hexdigest()
        if pdf_hash in seen_files:
            return f"Skipped (duplicate content): {pdf_url}"
        
        seen_files.add(pdf_hash)

        # Extract filename
        filename = os.path.basename(urllib.parse.urlparse(pdf_url).path)
        if not filename.lower().endswith('.pdf'):
            filename += '.pdf'
            
        save_path = os.path.join(save_dir, filename)
        
        # Save the file
        with open(save_path, 'wb') as f:
            f.write(pdf_content)
        
        return f"Downloaded: {filename}"

    except Exception as e:
        return f"Failed to download {pdf_url}: {str(e)}"


def download_pdfs(df, url_column, save_dir=r"C:\Users\amith\Kenpath\OFQUAL\cityandguilds_pdfs", max_workers=15):
    """Download PDFs from a DataFrame using threading."""
    os.makedirs(save_dir, exist_ok=True)
    
    seen_urls = set()
    seen_files = set()
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {
            executor.submit(download_pdf, row[url_column], save_dir, seen_urls, seen_files): row[url_column]
            for _, row in df.iterrows()
        }
        
        for future in as_completed(futures):
            print(future.result())

# Usage example
download_pdfs(pdf_df, 'pdf_url', max_workers=15)  # Adjust workers based on internet speed


Downloaded: 4520-barred-units-pdf.pdf
Downloaded: 0068_l3_dip_in_work_based_floristry_sfa_purpose-pdf.pdf
Downloaded: 0068_l2_dip_in_work_based_floristry_sfa_purpose-pdf.pdf
Downloaded: 4520-02_l2_diploma_in_ict_professional_competence-_qhb_v6-pdf.pdf
Downloaded: 4520-01_l1_certificate_qualification_handbook_v5-pdf.pdf
Downloaded: 50097957_l3_certificate_designing_and_planning_communications_networks_adult_purpose-pdf.pdf
Downloaded: 4520-03_l3_diploma_in-_ict_professional_competence-_qhb_v10_2-pdf.pdf
Downloaded: 4520-04_l4_diploma_in_ict_professional_competence-_qhb_v8,-d-,3-pdf.pdf
Downloaded: 4520_l1_l2_l3_l4_unit_handbook_units_301-380-pdf.pdf
Downloaded: 50117877_l4_diploma_ict_professional_competence_adult_purpose-pdf.pdf
Downloaded: 4520_l1_l2_l3_l4_unit_handbook_units_102-180-pdf.pdf
Downloaded: 4520_l1_l2_l3_l4_unit_handbook_units_201-280_v8_1-pdf.pdf
Downloaded: 50117889_l3_diploma_in_ict_professional_competence_purpose-pdf.pdf
Downloaded: 0068-31_l3_diploma_logbook_v3-pdf.p

In [6]:
pip install PyPDF2


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!pip install pikepdf

In [2]:
import os
import pikepdf

def delete_password_protected_pdfs(folder_path):
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(".pdf"):
            file_path = os.path.join(folder_path, filename)
            try:
                # Attempt to open the PDF without a password.
                with pikepdf.open(file_path):
                    pass  # If it opens, it is not password protected.
            except pikepdf.PasswordError:
                # If a PasswordError is raised, the PDF is password-protected.
                print(f"Deleting password-protected PDF: {filename}")
                os.remove(file_path)
            except pikepdf.PdfError:
                print(f"Error processing {filename}. It may be corrupted or not a valid PDF.")
            except Exception as e:
                print(f"An unexpected error occurred with {filename}: {e}")

folder_path = r"C:\Users\amith\Kenpath\OFQUAL\OFQUAL_27-02-2025\cityandguilds_pdfs"

if os.path.exists(folder_path) and os.path.isdir(folder_path):
    delete_password_protected_pdfs(folder_path)
    print("Finished processing PDFs.")
else:
    print("Invalid folder path.")

Deleting password-protected PDF: 0074-03_-assessment_pack_v3-4-pdf.pdf
Error processing 0076_need_to_know_december_2011_v1-pdf.pdf. It may be corrupted or not a valid PDF.
Error processing 0078_need_to_know_december_2011_v1-pdf.pdf. It may be corrupted or not a valid PDF.
Deleting password-protected PDF: 0172-001_2021_synoptic_assignment_-_with_adaptations_v1-1-pdf.pdf
Deleting password-protected PDF: 0172-001_2022_synoptic_assignment_v1-1-pdf.pdf
Deleting password-protected PDF: 0172-001_2023_synoptic_assignment_v1-1-pdf.pdf
Deleting password-protected PDF: 0172-001_2024_synoptic_assignment_v1-1-pdf.pdf
Deleting password-protected PDF: 0172-001_2025_synoptic_assignment_v1-1-pdf.pdf
Deleting password-protected PDF: 0172-003_2021_synoptic_assignment_-_with_adaptions_v1-2-pdf.pdf
Deleting password-protected PDF: 0172-003_2022-_synoptic_assignment_v1-1-pdf.pdf
Deleting password-protected PDF: 0172-003_2023_synoptic_assignment_v1-1-pdf.pdf
Deleting password-protected PDF: 0172-003_2024_syn

In [5]:
import os

# Set the directory where the files are located
folder_path = r"C:\Users\amith\Kenpath\OFQUAL\skillsandeducation_pdfs"

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    if "Case Study Only" in filename:  # Check if the string is in the filename
        file_path = os.path.join(folder_path, filename)
        try:
            print(f"Deleting file: {filename}")
            os.remove(file_path)  # Delete the file
        except Exception as e:
            print(f"Error deleting {filename}: {e}")

print("Completed deleting files with 'recognition' in their names.")


Deleting file: 2208-GD-1__Sept 2023 Road Haulage Case Study Only.pdf
Deleting file: 2208-GD-1__September 2024 Road Haulage Case Study Only.pdf
Deleting file: 2208-IA-1__June 2023 Road Haulage Case Study Only_1.1.pdf
Deleting file: 2208-IA-4__June 2024 Series P2 (Road Haulage) Case Study Only.pdf
Deleting file: 2208-IA-8__RH Case Study Only March 2024.pdf
Deleting file: 2209-IA-11__March 2024 Passenger Transport Case Study Only.pdf
Deleting file: 2209-IA-1__June 2022 Series P2 (Passenger Transport) Case Study Only.pdf
Deleting file: 2209-IA-1__June 2023 Passenger Transport Case Study Only_1.1.pdf
Deleting file: 2209-IA-1__March 2022 Series P2 (Passenger Transport) Case Study Only.pdf
Deleting file: 2209-IA-1__September 2022 Series P2 (Passenger Transport) Case Study Only.pdf
Deleting file: 2209-IA-5__June 2024 Passenger Transport Case Study Only.pdf
Completed deleting files with 'recognition' in their names.


In [1]:
print("Hello ")

Hello 
